In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#transformers은 버전 3으로 고정
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install konlpy

     |████████████████████████████████| 46.9MB 63kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 26.3MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595686 sha256=06896d2adf7f002e156c3c66f1470e1a5f53d02ec93546a9c087af6c5264cc88
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 31.8MB/s 
     |████████████████████████████████| 757kB 27.2MB/s 
     |████████████████████████████████| 3.0MB 49.9MB/s 
     |████████████████████████████████| 901kB 49.3MB/s 
     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 47.9MB/s 
     |████████████████████████████████| 92kB 13.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ugqfldoa
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ugqfldoa
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=9fdd16a7dc6866745e7a14e5920be7f6bb9052976134eb55c6a0f5fb582be57b
  Stored in directory: /tmp/pip-ephem-wheel-cache-esrx9jli/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
from torch import cuda
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시 cuda:0
#device = torch.device("cuda:0")
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training koBERT Model on {device}\n{"="*44}')

Training koBERT Model on cuda


In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd
import re
import csv
from sklearn.model_selection import train_test_split

In [ ]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2021-05-17 23:48:17--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2021-05-17 23:48:17--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucab2bfeb6a18afa31f362fa81ef.dl.dropboxusercontent.com/cd/0/get/BOuzV4arbDsf6V-M_Tq0Dpctx2tkIj9JIhXiCChhaodKdid_MsDZO6Iygt6otwDtlYfkdmR3TqgjYf87Gc-7t8IO9S0js92IdmkAwo9SvGErojyjbBn67HGjq7LO3m-O8F15eAJgSY7Nuo7O_51OaBDd/file?dl=1# [following]
--2021-05-17 23:48:18--  https://ucab2bfeb6a18afa31f362fa81ef.dl.dropboxusercontent.com/cd/0/get/BOuzV4arbDsf6V-M_Tq0Dpctx2tkIj9JIhXiCChhaodKdid_MsDZO6Iygt6otwDtlYfkdmR3T

In [ ]:
#TSVDataset
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [ ]:
#Tokenizer
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

###파라미터 세팅

In [ ]:
max_len = 128
batch_size = 64
warmup_ratio = 0.01
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7074747681617737 train acc 0.484375
epoch 1 batch id 201 loss 0.3817155063152313 train acc 0.6843905472636815
epoch 1 batch id 401 loss 0.4968573749065399 train acc 0.7516754987531172
epoch 1 batch id 601 loss 0.40345534682273865 train acc 0.7816139767054908
epoch 1 batch id 801 loss 0.372974157333374 train acc 0.7974797128589264
epoch 1 batch id 1001 loss 0.22701823711395264 train acc 0.8080981518481518
epoch 1 batch id 1201 loss 0.3944775462150574 train acc 0.8157264779350542
epoch 1 batch id 1401 loss 0.3631180226802826 train acc 0.8214110456816559
epoch 1 batch id 1601 loss 0.3839332163333893 train acc 0.8263878044971893
epoch 1 batch id 1801 loss 0.2628272771835327 train acc 0.8306583148250972
epoch 1 batch id 2001 loss 0.43210190534591675 train acc 0.8349106696651675
epoch 1 batch id 2201 loss 0.33359163999557495 train acc 0.8379571785552021

epoch 1 train acc 0.8402237094709898


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8775575447570333


...


epoch 3 test acc 0.8895660166240409


epoch 4 batch id 1 loss 0.443019300699234 train acc 0.84375
epoch 4 batch id 201 loss 0.0334237702190876 train acc 0.9408426616915423
epoch 4 batch id 401 loss 0.15222056210041046 train acc 0.947591957605985
epoch 4 batch id 601 loss 0.12717664241790771 train acc 0.9507591514143094
epoch 4 batch id 801 loss 0.17692963778972626 train acc 0.9504915730337079
epoch 4 batch id 1001 loss 0.1705377846956253 train acc 0.9510801698301699
epoch 4 batch id 1201 loss 0.06709609925746918 train acc 0.9519671107410491
epoch 4 batch id 1401 loss 0.06743423640727997 train acc 0.9520766416845111
epoch 4 batch id 1601 loss 0.1789131909608841 train acc 0.9523539975015616
epoch 4 batch id 1801 loss 0.039051637053489685 train acc 0.9534199750138812
epoch 4 batch id 2001 loss 0.13585743308067322 train acc 0.9539995627186407
epoch 4 batch id 2201 loss 0.2212992012500763 train acc 0.9543673330304407

epoch 4 train acc 0.9550625711035267



epoch 4 test acc 0.8909247122762148


epoch 5 batch id 1 loss 0.41746997833251953 train acc 0.859375
epoch 5 batch id 201 loss 0.024877168238162994 train acc 0.9611318407960199
epoch 5 batch id 401 loss 0.029744241386651993 train acc 0.965515897755611
epoch 5 batch id 601 loss 0.1421547383069992 train acc 0.9676320715474209
epoch 5 batch id 801 loss 0.22402366995811462 train acc 0.9661946004993758
epoch 5 batch id 1001 loss 0.10356265306472778 train acc 0.9663617632367633
epoch 5 batch id 1201 loss 0.02007022686302662 train acc 0.966473251457119
epoch 5 batch id 1401 loss 0.06075555831193924 train acc 0.9658948965024983
epoch 5 batch id 1601 loss 0.10102493315935135 train acc 0.9659782948157402
epoch 5 batch id 1801 loss 0.04863765090703964 train acc 0.9665637146029983
epoch 5 batch id 2001 loss 0.1499081552028656 train acc 0.9668759370314842
epoch 5 batch id 2201 loss 0.09129661321640015 train acc 0.9668403566560654

epoch 5 train acc 0.9671835004266212



epoch 5 test acc 0.8926630434782609


In [ ]:
PATH = '/content/drive/My Drive/'
torch.save(model.state_dict(), PATH + 'kobert_nsmc.pt')
#model_tc = torch.load(PATH + 'model_tc.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수

##예측코드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#transformers은 버전 3으로 고정
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install konlpy
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 46.9MB 108kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 2.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595688 sha256=8d4aa9ef180d5d8a8ec2735d08e44d5bd55de1cd33786cfccc492919380ba54b
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 2.9MB/s 
     |████████████████████████████████| 757kB 2.9MB/s 
     |████████████████████████████████| 3.0MB 13.7MB/s 
     |████████████████████████████████| 901kB 29.1MB/s 
     |████████████████████████████████| 19.4MB 50.8MB/s 
     |████████████████████████████████| 460kB 34.9MB/s 
     |████████████████████████████████| 92kB 9.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling b

In [3]:
import pandas as pd
news = pd.read_csv('/content/drive/MyDrive/data/news/news_all.csv', encoding = 'utf-8-sig')

In [4]:
import torch
from torch import nn
from torch import cuda
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"="*44}')

Training MNIST Model on cuda


In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [7]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return torch.softmax(self.classifier(out),1)

In [8]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [9]:
model.load_state_dict(torch.load("/content/drive/My Drive/kobert_nsmc.pt"))

<All keys matched successfully>

In [10]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

using cached model


In [11]:
news.head()

,date,title,media,reporter,content,summary
0,2021.05.02,0.59%p차…아슬아슬하게 '친문→쇄신' 택한 송영길 민주당,중앙일보,한영익,송영길 더불어민주당 신임 당대표가 2일 서울 여의도 중앙당사에서 열린 임시전국대의원...,여당인 더불어민주당의 전당대회에서 선출직 공직자인 더불어민주당 대표는 친문 핵심 홍...
1,2021.05.02,"국민의힘, 민주 송영길 대표 선출에 “기대보다 우려가 커”",중앙일보,이지영,윤희석 국민의힘 대변인. 연합뉴스 국민의힘은 더불어민주당 새 당대표로 송영길 의원이...,더불어민주당 새 당대표로 송영길 의원이 선출된 것에 대해 기대보다 우려가 크다는 반...
2,2021.05.02,"이재명, 민주 새 지도부에 “민생개혁 성과 보여줘야···경기도 함께하겠다”",중앙일보,이지영,이재명 경기도지사. 연합뉴스 이재명 경기도지사는 더불어민주당 송영길 신임 대표 체제...,이재명..연합뉴스 이재명..이지영.jiyoung2@joongang.co.kr to ...
3,2021.05.02,바이든 ‘모범답안’에 北 노골적 불쾌감…한·미 공조도 시험대,중앙일보,정진우,조 바이든 미국 대통령. AP=연합뉴스 조 바이든 미국 행정부가 새로운 대북정책을 ...,문재인 대통령이 한반도 평화 프로세스를 재가동하려 했으나 북ᆞ미 간에 벌써 비핵화 ...
4,2021.05.02,"16년 칼갈았다, '할말은 하는' 송영길 4수 끝 여당 당대표",중앙일보,김효성,송영길 민주당 대표는 2일 전당대회 정견 발표에서 ″위기임을 인정하고 그 지점에서 ...,여당인 민주당의 최고위원인 '동서남북 포럼'의 도움으로 2008년 최고위원에 세 차...


In [13]:
news = news.drop(['date','title','media','reporter','content'], axis=1)

In [14]:
news["content"] = news["summary"].apply(lambda x : ' '.join(x.strip().split()))
news = news.drop(['summary'], axis=1)
news["info"] = 1
news.head()

,content,info
0,여당인 더불어민주당의 전당대회에서 선출직 공직자인 더불어민주당 대표는 친문 핵심 홍...,1
1,더불어민주당 새 당대표로 송영길 의원이 선출된 것에 대해 기대보다 우려가 크다는 반...,1
2,이재명..연합뉴스 이재명..이지영.jiyoung2@joongang.co.kr to ...,1
3,문재인 대통령이 한반도 평화 프로세스를 재가동하려 했으나 북ᆞ미 간에 벌써 비핵화 ...,1
4,여당인 민주당의 최고위원인 '동서남북 포럼'의 도움으로 2008년 최고위원에 세 차...,1


In [15]:
news[["content","info"]].to_csv("/content/drive/My Drive/news_test.txt", sep='\t', index=False)
dataset_test = nlp.data.TSVDataset("/content/drive/My Drive/news_test.txt", field_indices=[0,1], num_discard_samples=1)
data_test =  BERTDataset(dataset_test, 0, 1, tok, 128, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=64, num_workers=2)

In [16]:
def model_pred(arg_list):
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    valid_length = valid_length
    segment_ids = segment_ids.long().to(device)
    label = label.long().to(device)
    pred_var = model(token_ids, valid_length, segment_ids)
    _, predict = torch.max(pred_var,1)
    # arg_list.extend(predict.tolist())
    arg_list.extend(pred_var.tolist())

In [17]:
model.eval()
list_pred = []
model_pred(list_pred)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [18]:
list_pred[:5]

[[0.008980143815279007, 0.9910197854042053],
 [0.18868497014045715, 0.8113150596618652],
 [0.04337434098124504, 0.9566256403923035],
 [0.9930417537689209, 0.006958244368433952],
 [0.03820670768618584, 0.9617932438850403]]

In [19]:
info = [0] * len(list_pred)
for i in range(len(list_pred)):
  info[i] = list_pred[i][1]

In [20]:
info[:5]

[0.9910197854042053,
 0.8113150596618652,
 0.9566256403923035,
 0.006958244368433952,
 0.9617932438850403]

In [21]:
news["info"] = info
news = news[["content","info"]]

In [22]:
news.head()

,content,info
0,여당인 더불어민주당의 전당대회에서 선출직 공직자인 더불어민주당 대표는 친문 핵심 홍...,0.991020
1,더불어민주당 새 당대표로 송영길 의원이 선출된 것에 대해 기대보다 우려가 크다는 반...,0.811315
2,이재명..연합뉴스 이재명..이지영.jiyoung2@joongang.co.kr to ...,0.956626
3,문재인 대통령이 한반도 평화 프로세스를 재가동하려 했으나 북ᆞ미 간에 벌써 비핵화 ...,0.006958
4,여당인 민주당의 최고위원인 '동서남북 포럼'의 도움으로 2008년 최고위원에 세 차...,0.961793


In [23]:
news.to_csv('/content/drive/My Drive/news_sentiment(score2).csv', index=False)